In [ ]:
import pandas as pd
import sklearn.feature_extraction as fe

#
import pickle
import json
import os

Notebook-only

In [3]:
#
original_directory = os.getcwd()
os.chdir("../../data/raw")
os.listdir()

['.ipynb_checkpoints',
 '20News_coarse.pkl',
 '20News_seedwords.json',
 'nyt_coarse.pkl',
 'nyt_seedwords.json']

In [4]:
# read data

#
with open("nyt_coarse.pkl", "rb") as fp:
    data_nyt = pickle.load(fp)
with open("20News_coarse.pkl", "rb") as fp:
    data_20news = pickle.load(fp)
    
with open("nyt_seedwords.json") as fp:
    seed_nyt = json.load(fp)
with open("20News_seedwords.json") as fp:
    seed_20news = json.load(fp)

In [58]:
#
data_nyt.head()
# data_nyt.iloc[0]["sentence"]

,sentence,label
0,"nasa, in preparation for a spacewalk on saturd...",science
1,if professional pride and strong defiance can ...,sports
2,"admittedly, the language is reconstructed and ...",arts
3,"palermo, sicily — roberta vinci beat top-seede...",sports
4,the argentine soccer club san lorenzo complete...,sports


In [6]:
#
data_20news.head() #newline characters visible

,sentence,label
0,from: (where's my thing)\nsubject: what car i...,rec
1,from: (guy kuo)\nsubject: si clock poll - fin...,comp
2,from: (thomas e willis)\nsubject: pb question...,comp
3,from: jgreen@amber (joe green)\nsubject: re: w...,comp
4,from: (jonathan mcdowell)\nsubject: re: shutt...,sci


In [7]:
#
seed_nyt #same format as 20News seed words

{'arts': ['music', 'orchestra', 'album', 'opera', 'ballet'],
 'business': ['companies', 'euro', 'economy', 'batteries', 'sales'],
 'science': ['space', 'researchers', 'scientists', 'research', 'science'],
 'sports': ['yankees', 'cup', 'league', 'basketball', 'golf'],
 'politics': ['republicans', 'senator', 'senate', 'democrats', 'election']}

# TF-IDF

## NYT

IR-TF-IDF treats the seed word set for each class as a query. The relevance of a document to a label is computed by aggregated TF-IDF values of its respective seed words. The label with the highest relevance is assigned to each document.

In [48]:
genres_nyt = {}
for genre,seed_words in seed_nyt.items():
    for seed_word in seed_words:
        genres_nyt[seed_word] = genre
        
genres_nyt

{'music': 'arts',
 'orchestra': 'arts',
 'album': 'arts',
 'opera': 'arts',
 'ballet': 'arts',
 'companies': 'business',
 'euro': 'business',
 'economy': 'business',
 'batteries': 'business',
 'sales': 'business',
 'space': 'science',
 'researchers': 'science',
 'scientists': 'science',
 'research': 'science',
 'science': 'science',
 'yankees': 'sports',
 'cup': 'sports',
 'league': 'sports',
 'basketball': 'sports',
 'golf': 'sports',
 'republicans': 'politics',
 'senator': 'politics',
 'senate': 'politics',
 'democrats': 'politics',
 'election': 'politics'}

In [11]:
# ir-tf-idf replication

# intialize vectorizer
tfidf_model = fe.text.TfidfVectorizer(input="content", stop_words = {'english'})
tfidf_vector = tfidf_model.fit_transform(data_nyt["sentence"])

In [67]:
#get vector with
filtered = list(filter(lambda item: item[1] in genres_nyt, zip(tfidf_vector[0].toarray()[0].tolist(),tfidf_model.get_feature_names_out())))

In [68]:
# list(filtered)
label_test = max(filtered, key=lambda item: item[0])
label_test[1]

'space'

In [33]:
# nyt_tfidf = dict(zip(tfidf_model.get_feature_names_out()[i],tfidf_vector[].toarray()[0].tolist()[i]) for i in range(
#     len(tfidf_model.get_feature_names_out())))
# nyt_tfidf

ValueError: dictionary update sequence element #1 has length 3; 2 is required

In [60]:
tfidf_model.get_feature_names_out()

array(['00', '000', '0000', ..., 'ötzl', 'özil', 'über'], dtype=object)

In [83]:
def ir_tfidf(data, seeds):
    '''
    Return a DataFrame of the input with predicted labels using IR-TF-IDF
    
    :param: data: a DataFrame with "sentence" and "label" attributes
    :param: seeds: a dictionary of labels (keys) with a list of seed words (values)
    '''
    #reverse seeds dictionary
    genres = {}
    for genre,seed_words in seed_nyt.items():
        for seed_word in seed_words:
            genres[seed_word] = genre
            
    data_ind = data.reset_index()
            
    model = fe.text.TfidfVectorizer(input="content", stop_words = {'english'})
    vector = tfidf_model.fit_transform(data["sentence"])
    features = tfidf_model.get_feature_names_out()
    
    def predict(ind):
        filtered = filter(lambda item: item[1] in genres, zip(vector[ind].toarray()[0].tolist(),features))
        prediction = max(filtered, key=lambda item: item[0])
        return prediction[1]
    
    df = data_ind.assign(prediction = data_ind["index"].apply(predict))
    return df

In [ ]:
tfidf_nyt = ir_tfidf(data_nyt,seed_nyt)

In [ ]:
tfidf_nyt